In [1]:
import numpy as np
import gmpy2 as gmp
from multiprocess import Pool, cpu_count
from hyperopt import hp, fmin, tpe
import xgboost as xgb
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
import csv
import dask.dataframe as dd

In [2]:
#
# Create a dataset of N-bit binary strings. Each string represents an integer labeled 0 (composite) or 1 (prime)
#
def create_N_bit_labeled_strings(N, chunk_size=1024):
    
    # data container
    data = []
    num_cores = cpu_count()

    # save data to a CSV file 
    csv_file = f'binary_data_{N}.csv'

    def is_prime(binary_string):
        decimal_value = gmp.mpz(binary_string, base=2)
        return binary_string, int(gmp.is_prime(decimal_value))

    with open(csv_file, "w", newline="") as file:

        writer = csv.writer(file)
        writer.writerow(["BinaryString", "Label"])

        # generate binary strings and check primality in chunks
        for chunk_start in range(0, 2**N, chunk_size):

            print(chunk_start)

            chunk_end = min(chunk_start + chunk_size, 2**N)

            binary_strings = [format(i, f'0{N}b') for i in range(chunk_start, chunk_end)]

            with Pool(num_cores) as pool:
                data = pool.map(is_prime, binary_strings)

            writer.writerows(data)

    print('All data saved to '+csv_file)

In [3]:
#
# Datasets are available for 18 and 24 bit strings
#

In [4]:
#
# Testing 18-bit strings first
#

In [5]:
# load data using Dask
ddf_18 = dd.read_csv('binary_data_18.csv', assume_missing=True, dtype={"BinaryString": str, "Label": int})

In [8]:
# primes vs composites
num_composites = sum(ddf_18['Label'] == 0)
num_primes     = sum(ddf_18['Label'] == 1)
print(f'{num_primes} primes and {num_composites} composites out of {len(ddf_18)} numbers in total')

23000 primes and 239144 composites out of 262144 numbers in total


In [9]:
#
# Try some educated guess of a model
#

In [10]:
# train / test split
X_train, X_test = ddf_18.random_split([0.8, 0.2], random_state=42)

# convert Dask dataframes to DMatrix XGBoost format
dtrain = xgb.DMatrix(X_train.drop(columns=['Label']), X_train['Label'])
dtest = xgb.DMatrix(X_test.drop(columns=['Label']), X_test['Label'])

# choose XGBoost hyperparameters
params = {
    'objective': 'binary:logistic',
    'scale_pos_weight': num_composites / num_primes,
    'eval_metric': 'auc',  # evaluation metric
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'max_depth': 10,
    'reg_alpha': 0.1,
    'reg_lambda': 1,
    'learning_rate': 0.05,
    'n_jobs': -1
}

# train model
model = xgb.train(params, dtrain, evals=[(dtrain, 'train'), (dtest, 'test')], verbose_eval=True)

# make predictions 
y_pred_prob = model.predict(dtest)

# convert probabilities to binary labels
threshold = y_pred_prob.mean()
y_pred = [1 if pred >= threshold else 0 for pred in y_pred_prob]

# true values
y_true = X_test['Label']

[0]	train-auc:0.53302	test-auc:0.50892
[1]	train-auc:0.53633	test-auc:0.50437
[2]	train-auc:0.53859	test-auc:0.50302
[3]	train-auc:0.53961	test-auc:0.50128
[4]	train-auc:0.54034	test-auc:0.50039
[5]	train-auc:0.54040	test-auc:0.50082
[6]	train-auc:0.54076	test-auc:0.50130
[7]	train-auc:0.54107	test-auc:0.49986
[8]	train-auc:0.54149	test-auc:0.49930
[9]	train-auc:0.54167	test-auc:0.49867


In [11]:
#
# Calculate and output confusion matrix
#
confusion = confusion_matrix(y_true, y_pred, normalize='all')
print("Confusion Matrix:")
print(np.array2string(confusion, formatter={'all': lambda x: f'{x:.6f}'}))

Confusion Matrix:
[[0.543471 0.368693]
 [0.052039 0.035797]]


In [12]:
#
# Now we do Bayesian hyperparameter optimization
#

In [14]:
def best_model_search(ddf):
    
    # train / test split
    X_train, X_test = ddf.random_split([0.8, 0.2], random_state=42)
    
    # primes vs composites
    num_composites = sum(ddf['Label'] == 0)
    num_primes     = sum(ddf['Label'] == 1)
    print(f'{num_primes} primes and {num_composites} composites out of {len(ddf)} numbers total')

    # convert Dask dataframes to DMatrix XGBoost format
    dtrain = xgb.DMatrix(X_train.drop(columns=['Label']), X_train['Label'])
    dtest = xgb.DMatrix(X_test.drop(columns=['Label']), X_test['Label'])
    
    # hyperparameter search space
    space = {
        'max_depth': hp.quniform('max_depth', 2, 16, 1),
        'scale_pos_weight': hp.uniform('scale_pos_weight', 0.5, 1),
        'learning_rate': hp.loguniform('learning_rate', -4, 0),
        'subsample': hp.uniform('subsample', 0.5, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
        'reg_alpha': hp.loguniform('reg_alpha', -5, 2),
        'reg_lambda': hp.loguniform('reg_lambda', -5, 2),
    }

    # objective function to optimize
    def objective(params):

        params['max_depth'] = int(params['max_depth'])
        params['objective'] = 'binary:logistic'
        params['eval_metric'] = 'auc'
        params['n_jobs'] = -1

        # Create DMatrix
        dtrain = xgb.DMatrix(X_train.drop(columns=['Label']), X_train['Label'])
        dtest = xgb.DMatrix(X_test.drop(columns=['Label']), X_test['Label'])

        # Train the model with given hyperparameters
        model = xgb.train(params, dtrain, evals=[(dtrain, 'train'), (dtest, 'test')],
                          verbose_eval=False, early_stopping_rounds=10)

        # Evaluate the model and return the negative AUC-PR as the loss (to minimize)
        y_pred_prob = model.predict(dtest)
        y_true = X_test['Label']
        auc_score = -roc_auc_score(y_true, y_pred_prob)

        return auc_score

    # use Hyperopt to find the best hyperparameters
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50)

    # record best hyperparameters
    best_max_depth = int(best['max_depth'])
    best_learning_rate = best['learning_rate']
    best_subsample = best['subsample']
    best_colsample_bytree = best['colsample_bytree']
    best_reg_alpha = best['reg_alpha']
    best_reg_lambda = best['reg_lambda']

    # train the best XGBoost model 
    best_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'aucpr',
        'max_depth': best_max_depth,
        'learning_rate': best_learning_rate,
        'subsample': best_subsample,
        'colsample_bytree': best_colsample_bytree,
        'reg_alpha': best_reg_alpha,
        'reg_lambda': best_reg_lambda,
        'n_jobs': -1
    }

    best_model = xgb.train(best_params, dtrain, evals=[(dtrain, 'train'), (dtest, 'test')],
                            verbose_eval=True)

    # make predictions with the best model
    y_pred_prob_best = best_model.predict(dtest)

    # convert probabilities into bits
    threshold = y_pred_prob_best.mean()
    y_pred = [1 if pred >= threshold else 0 for pred in y_pred_prob_best]

    # ground truth
    y_true = X_test['Label']

    # calculate and output confusion matrix
    confusion = confusion_matrix(y_true, y_pred, normalize='all')
    print("Best Model Confusion Matrix:")
    print(np.array2string(confusion, formatter={'all': lambda x: f'{x:.6f}'}))

In [15]:
#
# Hyperparameter search for 18-bit strings first
#
best_model_search(ddf_18)

23000 primes and 239144 composites out of 262144 numbers total
100%|████████| 50/50 [01:01<00:00,  1.24s/trial, best loss: -0.5294763465873017]
[0]	train-aucpr:0.09783	test-aucpr:0.09873
[1]	train-aucpr:0.09843	test-aucpr:0.09894
[2]	train-aucpr:0.09867	test-aucpr:0.09865
[3]	train-aucpr:0.09871	test-aucpr:0.09878
[4]	train-aucpr:0.09895	test-aucpr:0.09864
[5]	train-aucpr:0.09906	test-aucpr:0.09878
[6]	train-aucpr:0.09914	test-aucpr:0.09883
[7]	train-aucpr:0.09919	test-aucpr:0.09857
[8]	train-aucpr:0.09931	test-aucpr:0.09846
[9]	train-aucpr:0.09935	test-aucpr:0.09840
Best Model Confusion Matrix:
[[0.638886 0.273278]
 [0.058209 0.029628]]


In [18]:
#
# The confusion matrix is composed as C(true_label, predicted_label), so that 
# C(0,0) are true negatives, 
# C(0,1) are false positives, 
# C(1,0) are false negatives,
# C(1,1) are true positves.
#
# The true positive rate is ~ 0.03, which is of course *very* low, but this is exactly what we expected
#

In [17]:
#
# Let's try 24-bit strings
#

In [18]:
# load data using Dask
ddf_24 = dd.read_csv('binary_data_24.csv', assume_missing=True, dtype={"BinaryString": str, "Label": int})

In [19]:
#
# Hyperparameter search for 24-bit strings 
#
best_model_search(ddf_24)

1077871 primes and 15699345 composites out of 16777216 numbers total
100%|███████| 50/50 [3:08:27<00:00, 226.16s/trial, best loss: -0.51854879458775]
[0]	train-aucpr:0.06937	test-aucpr:0.06941
[1]	train-aucpr:0.06939	test-aucpr:0.06945
[2]	train-aucpr:0.06940	test-aucpr:0.06946
[3]	train-aucpr:0.06940	test-aucpr:0.06946
[4]	train-aucpr:0.06941	test-aucpr:0.06946
[5]	train-aucpr:0.06941	test-aucpr:0.06946
[6]	train-aucpr:0.06941	test-aucpr:0.06946
[7]	train-aucpr:0.06941	test-aucpr:0.06946
[8]	train-aucpr:0.06941	test-aucpr:0.06946
[9]	train-aucpr:0.06941	test-aucpr:0.06947
Best Model Confusion Matrix:
[[0.635707 0.300054]
 [0.041912 0.022326]]


In [20]:
#
# The true positive rate is ~ 0.02, which is even lower than before
#